In [18]:
from random import randint
from time import sleep
import datetime
import yfinance as yf
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline
from mystockrec import xlsx_df

In [23]:
df = xlsx_df('df_k_687_2021.10.06 01_34_53.xlsx')
display(df.loc[0:3,'Unnamed: 0'])
df.head()

0    1
1    3
2    4
3    5
Name: Unnamed: 0, dtype: int64

,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
0,1,DUERR AG O.N.,DUE.DE,EUR,0.01104,0.091,0.20667,0.04322,125.410960,1.750285,6
1,3,COMPUGROUP MED. NA O.N.,COP.DE,EUR,0.04650,0.316,0.37123,0.17535,52.999240,0.836374,0
2,4,NFON AG INH O.N.,NFN.DE,EUR,-0.01891,0.160,0.38878,0.01379,52.999240,1.646321,6
3,5,MEDARTIS N,MED.SW,CHF,0.02513,0.291,0.83734,0.14498,105.835800,0.771998,2
4,6,SAP SE,SAP.DE,EUR,0.06056,-0.011,0.71777,0.26134,23.425348,0.927594,2


In [24]:
#clean data set: in "df_spotify" and "df" we lower cases for song title and clean it from special characters.
df['shortName'] = df['shortName'].apply(lambda x: x.capitalize().translate({ord(c): "" for c in "!?/[]|{}.:()'\""}))

In [26]:
def get_input(): 
    shortName_input = input("What company do you want to check? ")
    shortName_input = shortName_input.capitalize()
    comp_match = df[df['shortName'].str.contains(shortName_input)]
    if len(comp_match) > 0:
        print("The following company was found in the database, let's take a closer look.")
        display(comp_match)
    return shortName_input

In [27]:
get_input()

What company do you want to check? apple
The following company was found in the database, let's take a closer look.


,Unnamed: 0,shortName,symbol,financialCurrency,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,trailingPE,beta,cluster
315,342,Apple inc,AAPL,USD,0.19302,0.364,0.41005,0.31955,27.222006,1.222222,5


'Apple'

In [ ]:
#Check if song is in large DB/ if input makes any sense
def comp_check():
    comp_check = df_spotify[df_spotify['track_name'].str.contains(song_input)]
    if len(song_check0) > 0:
        return song_check0 
    elif len(comp_match) > 1:
    else:
        get_input()
song_check0 = song_check_overall()

In [ ]:
# df['symbol']
# user input = 'AAPL'
# df.loc[df['symbol']==user input]
# if len(df.loc[df['symbol']==user input]) ==1
#    print('The following company was found:')
#    elif len(df.loc[df['symbol']==user input]) > 1
# df = df.loc

In [ ]:
# Selection idea
def user_selection():
    for index, item in enumerate(options):
                print(f"Option {index + 1} - {item}")
            while True:
                try: # Is used to avoid errors
                    choice = int(input("Selection: "))
                    if choice >= 1 and choice <= len(options):
                        return options[choice-1]
                except:
                    continue